In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
plt.style.use(style='seaborn')
%matplotlib inline

In [2]:
df=pd.read_csv(r'C:\Users\Rajavardhan\Downloads\HistoricalData.csv')

In [3]:
df.head()

,Date,Close/Last,Volume,Open,High,Low
0,07/09/2021,$277.94,23916670,$275.72,$278.05,$275.32
1,07/08/2021,$277.42,24618590,$276.9,$278.73,$274.87
2,07/07/2021,$279.93,23260030,$279.4,$280.6945,$277.15
3,07/06/2021,$277.66,31565560,$278.03,$279.3698,$274.3
4,07/02/2021,$277.65,26474410,$272.82,$278,$272.5


In [4]:
# Converting the date into datetime foramt
df["Date"]= pd.to_datetime(df["Date"])


In [5]:
df.head()

,Date,Close/Last,Volume,Open,High,Low
0,2021-07-09,$277.94,23916670,$275.72,$278.05,$275.32
1,2021-07-08,$277.42,24618590,$276.9,$278.73,$274.87
2,2021-07-07,$279.93,23260030,$279.4,$280.6945,$277.15
3,2021-07-06,$277.66,31565560,$278.03,$279.3698,$274.3
4,2021-07-02,$277.65,26474410,$272.82,$278,$272.5


In [6]:
# Renaming the Column
df.rename(columns = {"Close/Last": "Close"}, 
          inplace = True)


In [7]:
# Removing curency sign from the data 
df['Close'] = df['Close'].str.replace('$', '')


<ipython-input-7-7b6196c18eda>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Close'] = df['Close'].str.replace('$', '')


In [8]:
df=df[['Close']].copy()

In [9]:
df.head()

,Close
0,277.94
1,277.42
2,279.93
3,277.66
4,277.65


### Transforming this to a supervised learning problem

In [10]:
df['target']=df.Close.shift(-1)
df.dropna(inplace=True)
df.head()

,Close,target
0,277.94,277.42
1,277.42,279.93
2,279.93,277.66
3,277.66,277.65
4,277.65,271.6


### Train, Test split

In [1]:
def train_test_split(data, perc):
    data=data.values
    n= int(len(data)*(1-perc))
    return data[:n],data[n:]

In [12]:
train, test=train_test_split(df, 0.2)

In [13]:
print(len(df))
print(len(train))
print(len(test))

250
200
50


In [14]:
x=train[:,:-1]
y=train[:,-1]

In [22]:
y

array(['277.42', '279.93', '277.66', '277.65', '271.6', '270.9', '271.4',
       '268.72', '265.02', '266.69', '265.275', '265.51', '262.63',
       '259.43', '260.9', '257.38', '258.36', '259.89', '257.89',
       '257.24', '253.59', '252.57', '253.81', '250.79', '245.71',
       '247.3', '247.4', '249.68', '249.31', '251.49', '251.72', '250.78',
       '245.17', '246.48', '243.12', '243.08', '245.18', '248.15',
       '243.03', '239', '246.23', '247.18', '252.46', '249.73', '246.47',
       '247.79', '251.86', '252.18', '252.51', '254.56', '261.97',
       '261.55', '261.15', '257.17', '260.58', '258.26', '258.74',
       '260.74', '259.5', '255.59', '258.49', '255.91', '255.85',
       '253.25', '249.9', '247.86', '249.07', '242.35', '235.77',
       '231.85', '235.24', '236.48', '232.34', '235.46', '237.58',
       '235.99', '230.35', '230.72', '237.04', '237.71', '234.81',
       '235.75', '237.13', '232.42', '233.78', '227.39', '231.6',
       '226.73', '227.56', '233.87', '236.9

In [16]:
from xgboost import XGBRegressor

In [17]:
model=XGBRegressor(objective='reg:squarederror',n_estimators=1000)
model.fit(x,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
test[0]

array(['207.42', '202.54'], dtype=object)

In [19]:
val=np.array(test[0])
pred = model.predict(val)
print(pred[0])

209.43782


### Predict

In [20]:
def xgb_predict(train,val):
    train=np.array(train)
    x,y=train[:,:-1],train[:,-1]
    model=XGBRegressor()
    model.fit(x,y)
    
    val=np.array(val).reshape(-1,1)
    pred=model.predict(val)
    return pred[0]

In [21]:
print(pred)

[209.43782 204.71951]
